In [12]:
#updated
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import base64
import pandas as pd
import plotly.express as px

# Configure the connection to your MongoDB using the CRUD module
from animal_shelter2 import AnimalShelter

# Setup MongoDB connection
username = "aacuser"
password = "snhu1234"
db = AnimalShelter(username, password, 'nv-desktop-services.apporto.com', 32078, 'AAC', 'animals')

# Fetch data from MongoDB
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

# Setup the JupyterDash app
app = JupyterDash(__name__)

# Encode the Grazioso Salvare logo image
image_filename = 'Grazioso Salvare Logo.png'  # Ensure this file is available
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('utf-8')

# Layout of the dashboard
app.layout = html.Div([
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    #Logo with anchor tag
    html.Center(html.A(href='https://www.snhu.edu', children=[
        html.Img(src='data:image/png;base64,{}'.format(encoded_image))
    ])),
    html.H3("Dashboard by Jose Lara Hernandez"),
    html.Hr(),

    # Radio buttons for filtering
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water_rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain_rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster_tracking'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block'}
        )
    ),
    html.Hr(),

    # DataTable
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    
    
    html.Br(),
    html.Hr(),

    # Charts
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Callback to filter the DataTable based on the selected rescue type
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'water_rescue':
        filtered_df = pd.DataFrame.from_records(db.read({"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}}))
    elif filter_type == 'mountain_rescue':
        filtered_df = pd.DataFrame.from_records(db.read({"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}}))
    elif filter_type == 'disaster_tracking':
        filtered_df = pd.DataFrame.from_records(db.read({"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}}))
    else:
        filtered_df = pd.DataFrame.from_records(db.read({}))
        
    for column in filtered_df.columns:
        filtered_df[column] = filtered_df[column].apply(lambda x: str(x) if isinstance(x, (dict, list, object)) else x)
    
    return filtered_df.to_dict('records')

# Callback to update the pie chart based on the filtered data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Preferred Breeds by Rescue Type')
        )
    ]

# Callback to highlight the selected columns in the DataTable
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback to update the map based on the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    
    if index is None or len(index) == 0:
        row = 0
    else: 
        row = index[0]
        
    lat = float(dff.iloc[row, 13])
    lon = float(dff.iloc[row, 14])
    
    if not (-90 <= lat <= 90 and -180 <= lon <= 180):
        return []
    
    # Assume Austin, TX as a default location
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

# Run the Dash app
app.run_server(debug=True)


Dash app running on http://127.0.0.1:9986/
